In [1]:
import pandas as pd # allows us to work with tabular data
import requests # to send the API requests to Elsevier
import json # to read the JSON data that is returned by the APIs
import pickle # pickle files are a good way to save data for reuse in Python
from datetime import datetime

In [2]:
api_key = 'your api key'

In [3]:
'''search parameters: add values to the dictionary key:value pairs below
leave any fields that you don't want to use as either 0 or '''

search_string_parameters = {
    'ISSN' : '', # Limit results to results from a journal ISSN (add as a string). Leave = '' if no value.
    'AUTHOR-NAME' : '', # Search the author name field. lastname, firstname. e.g., Noble, Safiya
    'KEY' : '', # Add a keyword to search in the text of the article
    'PUBLISHER' : '', # Add a publisher name. e.g., Springer
    'EXACTSRCTITLE' : 'journal', # Add keywords that appear in the journal, book, or conference title. e.g., Informatics
    'AFFIL' : '', # add keywords that appear in the author's institutional affiliation. e.g., to find the University of Minnesota, search for University and Minnesota
    'TITLE' : '6G', # Add keywords that appear in the article or chapter title. Can use AND, OR, and AND NOT. e.g. cat AND dog 
    'OPENACCESS' : '1', # Add 1 to limit your results to open access articles or 0 for articles that are not open access
    'start_year' : 2013, # Limit your search to items published after this year. Leave = 0 if no value. YYYY , e.g., 1995
    'end_year' : 2024 # Limit your search to items published before this year. Leave = 0 if no value. YYYY, e.g., 2020
}
def search_builder(search_string_parameters):
    ''' Concatenates a search string query formatted for the Scopus Search API.
    - search_string_parameters expects a python dictionary with keys aligned to API search fields.
    '''
    search_string = ''

    for k,v in search_string_parameters.items():
        if k == 'start_year' and v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'PUBYEAR > {v}'
        elif k == 'end_year' and v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'PUBYEAR < {v}'
        elif v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'{k}({v})'

    return search_string

def create_url(search_string):
    '''Accepts a formatted search string that will be added to the Scopus Search API URL. 
    Requires a global api_key variable.
    Formats and returns a URL to send to the Scopus Search API.'''
    
    query = f'{search_string}'
    url_template = 'https://api.elsevier.com/content/search/scopus?query={query}&apiKey={api_key}'
    full_url = url_template.format(query=query, api_key=api_key)
    return full_url

def connect_to_endpoint(full_url, params={'cursor': '*'}, next_ = '*'):
    '''Accepts API URL with ISSN, default parameters, and next page cursor;
    Sends request to Scopus API and collects JSON results for each call;
    Returns r.json() for the ['search-results'] key.'''
    
    params['cursor'] = next_
    r = requests.get(full_url, params=params)
    r.raise_for_status()
    return r.json()['search-results'], r.headers

In [4]:
search_string = search_builder(search_string_parameters)
print(search_string)
full_url = create_url(search_string)
r_json, r_headers = connect_to_endpoint(full_url)

EXACTSRCTITLE(journal) AND TITLE(6G) AND OPENACCESS(1) AND PUBYEAR > 2013 AND PUBYEAR < 2024


In [5]:
print('Title:', r_json['entry'][0]['dc:title'], 
        '\nCreator:', r_json['entry'][0]['dc:creator'], 
        '\nPublication:', r_json['entry'][0]['prism:publicationName'],
        '\nDate:', r_json['entry'][0]['prism:coverDate'],
        '\nISSN:', r_json['entry'][0]['prism:eIssn'], 
        '\nDOI:', r_json['entry'][0]['prism:doi'],
        '\nAbstract', r_json['entry'][0])

Title: Electric Vehicles Energy Management for Vehicle-to-Grid 6G-Based Smart Grid Networks 
Creator: Naja R. 
Publication: Journal of Sensor and Actuator Networks 
Date: 2023-12-01 
ISSN: 22242708 
DOI: 10.3390/jsan12060079 
Abstract {'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85180644256'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85180644256?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85180644256&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85180644256&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85180644256', 'dc:identifier': 'SCOPUS_ID:85180644256', 'eid': '2-s2.0-85180644256', 'dc:title': 'Electric Vehicles Energy Manag

In [6]:
dois = [entry.get('prism:doi') for entry in r_json.get('entry', [])]
print(dois[0])

10.3390/jsan12060079


In [7]:
def create_url(doi):
    """Accepts a DOI as a string that will be added to the Abstract Retrieval API URL. 
    Requires a global api_key variable.
    Formats and returns a URL to send to the Abstract Retrieval API.
    """
    doi = f'{doi}'
    url_template = 'https://api.elsevier.com/content/abstract/doi/{doi}?&apiKey={api_key}'
    full_url = url_template.format(doi=doi, api_key=api_key)
    return full_url

def connect_to_endpoint(full_url):
    '''Input full_url from create_url function;
    Send request to Scopus Abstract Retrieval API
    Returns r.json response;'''
    
    r = requests.get(full_url, headers =  {'Accept': 'application/json'})
    r.raise_for_status()
    return r.json(), r.headers

In [8]:
full_url = create_url(dois[0])

In [9]:
r_json, r_headers = connect_to_endpoint(full_url)
r_json = r_json['abstracts-retrieval-response']
print(r_json['coredata']['dc:description'])

This research proposes a unique platform for energy management optimization in smart grids, based on 6G technologies. The proposed platform, applied on a virtual power plant, includes algorithms that take into account different profiles of loads and fairly schedules energy according to loads priorities and compensates for the intermittent nature of renewable energy sources. Moreover, we develop a bidirectional energy transition mechanism towards a fleet of intelligent vehicles by adopting vehicle-to-grid technology and peak clipping. Performance analysis shows that the proposed energy provides fairness to electrical vehicles, satisfies urgent loads, and optimizes smart grids energy.


In [10]:
from langchain_community.chat_models import ChatOpenAI

In [11]:
chat_model = ChatOpenAI(openai_api_key="your_openai_api_key")

C:\Users\user\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [12]:
abstract = r_json['coredata']['dc:description']
result = chat_model.predict(abstract + " 이 내용을 250자 내외로 요약해줘")
print(result)

C:\Users\user\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


이 연구는 6G 기술을 기반으로 한 스마트 그리드의 에너지 관리 최적화를 위한 독특한 플랫폼을 제안한다. 가상 발전소에 적용된 이 플랫폼은 다양한 부하 프로파일을 고려하는 알고리즘을 포함하며, 부하 우선순위에 따라 에너지를 공정하게 일정하게 하고 재생 에너지의 간헐적인 특성을 보상한다. 게다가, 우리는 차량간 그리드 기술과 피크 클리핑을 채택하여 지능형 차량의 플리트를 대상으로 양방향 에너지 전환 메커니즘을 개발한다. 성능 분석 결과, 제안된 에너지는 전기 차량에 공정성을 제공하고 긴급한 부하를 충족시키며 스마트 그리드 에너지를 최적화한다.
